In [ ]:
import pandas as pd
import pandas as pd
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np

df = pd.read_csv(r'CLPM_data.csv')

# Aggregate by 'YearMonth', 'Participant_Number', and 'Question', taking the mean score
df_aggregated = df.groupby(['YearMonth', 'Participant_Number', 'Question'], as_index=False)['Score'].mean()

# Aggregate scores by test (mean score for each test: BDI, LSAS, CFS)
df_aggregated['Test'] = df_aggregated['Question'].str.extract(r'([A-Za-z]+)')
df_mean_scores = df_aggregated.groupby(['YearMonth', 'Participant_Number', 'Test'], as_index=False)['Score'].mean()

# Pivot the dataset to wide format for CLPM
df_wide = df_mean_scores.pivot_table(
    index='Participant_Number',
    columns=['YearMonth', 'Test'],
    values='Score'
)

# Ensure the columns are sorted chronologically by converting YearMonth to Period objects
sorted_columns = sorted(df_wide.columns, key=lambda x: (pd.Period(x[0], freq='M').year, pd.Period(x[0], freq='M').month))
df_wide = df_wide[sorted_columns]

# Display a preview of the wide-format dataset
df_wide.head()

# CLPM Analysis Setup
from statsmodels.tsa.vector_ar.var_model import VAR

# Drop rows with missing values (if any) to ensure VAR can run
clpm_data = df_wide.dropna()

# Fit a Vector Autoregression (VAR) model as a basic CLPM
model = VAR(clpm_data)
results = model.fit(maxlags=2)

# Display results summary
print(results.summary())

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 19, Feb, 2025
Time:                     17:04:05
--------------------------------------------------------------------
No. of Equations:         9.00000    BIC:                   -23.9315
Nobs:                     275.000    HQIC:                  -25.2779
Log likelihood:           258.936    FPE:                4.27411e-12
AIC:                     -26.1804    Det(Omega_mle):     2.34268e-12
--------------------------------------------------------------------
Results for equation 2020-01_BDI
                     coefficient       std. error           t-stat            prob
----------------------------------------------------------------------------------
const                   0.225349         0.262864            0.857           0.391
L1.2020-01_BDI         -0.010072         0.082723           -0.122           0.903
L1.2020-01_CFS          0.026029         0.066

c:\Users\MadiL\Thesis CodeBase\thesisenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)


In [4]:
# Plotting significant cross-lagged effects (p-value < 0.05)
significant_effects = []
for i, equation in enumerate(results.params.columns):
    for j, param in enumerate(results.params.index):
        if results.pvalues.iloc[j, i] < 0.05:
            significant_effects.append((param, equation, results.params.iloc[j, i]))

print("\nSignificant Cross-Lagged Effects:")
for effect in significant_effects:
    print(f"{effect[0]} -> {effect[1]}: Coefficient = {effect[2]:.4f}")



Significant Cross-Lagged Effects:
L1.2020-05_LSAS -> 2020-01_BDI: Coefficient = -0.0620
const -> 2020-01_CFS: Coefficient = -1.3815
const -> 2020-05_CFS: Coefficient = -1.6519
L2.2020-05_LSAS -> 2020-05_CFS: Coefficient = -0.0912
L2.2020-06_CFS -> 2020-05_LSAS: Coefficient = -0.2761
const -> 2020-06_CFS: Coefficient = -1.5197
L2.2020-01_CFS -> 2020-06_LSAS: Coefficient = -0.3165


In [5]:
# Aggregate by 'YearMonth', 'Participant_Number', and 'Question', taking the mean score
df_aggregated = df.groupby(['YearMonth', 'Participant_Number', 'Question'], as_index=False)['Score'].mean()

# Aggregate scores by test (mean score for each test: BDI, LSAS, CFS)
df_aggregated['Test'] = df_aggregated['Question'].str.extract(r'([A-Za-z]+)')
df_mean_scores = df_aggregated.groupby(['YearMonth', 'Participant_Number', 'Test'], as_index=False)['Score'].mean()

# Pivot the dataset to wide format for CLPM
df_wide = df_mean_scores.pivot_table(
    index='Participant_Number',
    columns=['YearMonth', 'Test'],
    values='Score'
)

# Ensure the columns are sorted chronologically by converting YearMonth to Period objects
sorted_columns = sorted(df_wide.columns, key=lambda x: (pd.Period(x[0], freq='M').year, pd.Period(x[0], freq='M').month))
df_wide = df_wide[sorted_columns]

# Display a preview of the wide-format dataset
df_wide.head()

# CLPM Analysis Setup

# Drop rows with missing values (if any) to ensure VAR can run
clpm_data = df_wide.dropna()

# Compute Variance Inflation Factor (VIF) to check for multicollinearity
vif_data = pd.DataFrame()
vif_data['Variable'] = clpm_data.columns.to_list()
vif_data['VIF'] = [variance_inflation_factor(clpm_data.values, i) for i in range(clpm_data.shape[1])]

print("Variance Inflation Factor (VIF) for each variable:")
print(vif_data)

# Fit a Vector Autoregression (VAR) model as a basic CLPM
model = VAR(clpm_data)
results = model.fit(maxlags=2)

# Display results summary
print(results.summary())

# Plotting significant cross-lagged effects (p-value < 0.05)
significant_effects = []
for i, equation in enumerate(results.params.columns):
    for j, param in enumerate(results.params.index):
        if results.pvalues.iloc[j, i] < 0.05:
            significant_effects.append((param, equation, results.params.iloc[j, i]))

print("\nSignificant Cross-Lagged Effects:")
for effect in significant_effects:
    print(f"{effect[0]} -> {effect[1]}: Coefficient = {effect[2]:.4f}")


Variance Inflation Factor (VIF) for each variable:
          Variable        VIF
0   (2020-01, BDI)   3.426907
1   (2020-01, CFS)  54.751372
2  (2020-01, LSAS)   6.971961
3   (2020-05, BDI)   2.078269
4   (2020-05, CFS)  48.961597
5  (2020-05, LSAS)   5.525473
6   (2020-06, BDI)   2.184995
7   (2020-06, CFS)  54.681892
8  (2020-06, LSAS)   4.755828
  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 19, Feb, 2025
Time:                     17:04:05
--------------------------------------------------------------------
No. of Equations:         9.00000    BIC:                   -23.9315
Nobs:                     275.000    HQIC:                  -25.2779
Log likelihood:           258.936    FPE:                4.27411e-12
AIC:                     -26.1804    Det(Omega_mle):     2.34268e-12
--------------------------------------------------------------------
Results for equation 2020-01_BDI
                     coeffi

c:\Users\MadiL\Thesis CodeBase\thesisenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
